# Setup

This notebook uses syntax that is not available in Python versions prior to 3.10. Follow the instructions below to set up an environment that will run this notebook.
1. Create a new `conda` environment.
```
$ conda create -n py310 python=3.10
```
This may not work if you are using a version of `conda` prior to 4.11. If you do not have `conda`, or need to upgrade, you can [download the latest installer](https://docs.conda.io/en/latest/miniconda.html). If you already have a recent version on your system, but not Python 3.10, then you can run:
```
$ conda install -c conda-forge python=3.10
```
After installing Python 3.10 (or if you already have it), create a new environment as described above. You can use the environment by running:
```
$ conda activate py310
```
2. In the new environment, install the `z3-solver` package.
```
(py310)$ pip install z3-solver
```
3. Install Jupyter notebook.
```
(py310)$ pip install notebook
```
You can run this notebook by navigating to the directory that you downloaded it to in a command line, and running:
```
(py310)$ jupyter notebook
```
You can navigate to the URL printed by that command in your browser, and opening `316-z3-live.ipynb`.

### A note for Windows users

You should be able to follow the instructions above in either of Windows' native shells, `cmd.exe` and Powershell. However, the course staff has not recently tested this path; we will try to help you address issues that arise, but our ability to do so may be limited depending on your system configuration.

There are two alternatives that you should consider using.
1. If you are running Windows 10 version 2004 and higher (Build 19041 and higher) or Windows 11, then you can run all of these commands from the Windows Subsystem for Linux (WSL2). See [the documentation](https://learn.microsoft.com/en-us/windows/wsl/install) for setting this up. This notebook was developed in Windows using the Ubuntu distribution available in the [Windows App store](https://apps.microsoft.com/store/detail/ubuntu/9PDXGNCFSCZV), so if you are having trouble getting set up natively, then this is the most likely route to success.
2. Use the Andrew linux cluster, where Python 3.10 is already installed as `python3`. You can install the `z3-python` and `notebook` packages with `pip`. To connect to a Jupyter notebook, set up an SSH tunnel when you connect to the cluster:
```
ssh -L 8888:127.0.0.1:8888 linux.andrew.cmu.edu
```
When you connect and run `jupyter notebook`, you can then paste the URL displayed by `jupyter` into your local browser.

In [1]:
from z3 import *
from __future__ import annotations
from dataclasses import dataclass

# Syntactic Definitions

Begin by defining the space of terms, formulas, and programs. The formula constructors are suffixed with `C` (for constants) and `F` (formulas) to avoid namespace collisions with Python with `z3`.

In [2]:
@dataclass
class Const:
    value: int
@dataclass
class Var:
    name: str
@dataclass
class Sum:
    left: Term
    right: Term
@dataclass
class Difference:
    left: Term
    right: Term

Term = Const | Var | Sum | Difference

In [3]:
@dataclass
class TrueC:
    _: None
@dataclass
class FalseC:
    _: None
@dataclass
class LtF:
    left: Term
    right: Term
@dataclass
class EqF:
    left: Term
    right: Term
@dataclass
class NotF:
    q: Formula
@dataclass
class AndF:
    p: Formula
    q: Formula
@dataclass
class OrF:
    p: Formula
    q: Formula
@dataclass
class ImpliesF:
    p: Formula
    q: Formula

Formula = TrueC | FalseC | LtF | EqF | NotF | AndF | OrF | ImpliesF

In [4]:
@dataclass
class Asgn:
    left: Var
    right: Term
@dataclass
class Assert:
    q: Formula
@dataclass
class Seq:
    alpha: Prog
    beta: Prog
@dataclass
class If:
    q: Formula
    alpha: Prog
    beta: Prog
@dataclass
class While:
    q: Formula
    alpha: Prog

Prog = Asgn | Assert | Seq | If | While

The following utility functions encode terms and formulas built using the constructors above as Z3 terms and formulas.

In [5]:
def term_enc(e: Term) -> IntNumRef:
    match e:
        case Const(value):
            return IntVal(value)
        case Var(name):
            return Int(name)
        case Sum(left, right):
            return term_enc(left) + term_enc(right)
        case Difference(left, right):
            return term_enc(left) - term_enc(right)            
        
def fmla_enc(p: Formula) -> BoolRef:
    match p:
        case TrueC(_):
            return BoolVal(True)
        case FalseC(_):
            return BoolVal(False)
        case LtF(left, right):
            return term_enc(left) < term_enc(right)
        case EqF(left, right):
            return term_enc(left) == term_enc(right)
        case NotF(p):
            return Not(fmla_enc(p))
        case AndF(p, q):
            return And(fmla_enc(p), fmla_enc(q))
        case OrF(p, q):
            return Or(fmla_enc(p), fmla_enc(q))
        case ImpliesF(p, q):
            return Implies(fmla_enc(p), fmla_enc(q))

The following helper provides a way to examine a program or formula object that is easier to interpret.

In [6]:
def prettyprint(o: Term | Formula | Prog, indent: int=0) -> str:
    ids = indent*' '
    
    if isinstance(o, Term):
        match o:
            case Const(value):
                return '{}'.format(value)
            case Var(name):
                return '{}'.format(name)
            case Sum(left, right):
                return '{} + {}'.format(
                    prettyprint(left),
                    prettyprint(right))
            case Difference(left, right):
                return '{} - {}'.format(
                    prettyprint(left),
                    prettyprint(right))
            
    elif isinstance(o, Formula):
        match o:
            case TrueC(_):
                return "true"
            case FalseC(_):
                return "false"
            case LtF(left, right):
                return '{} < {}'.format(
                    prettyprint(left),
                    prettyprint(right))
            case EqF(left, right):
                return '{} == {}'.format(
                    prettyprint(left),
                    prettyprint(right))
            case NotF(p):
                return 'not({})'.format(
                    prettyprint(p))
            case AndF(p, q):
                return '{} && {}'.format(
                    prettyprint(p),
                    prettyprint(q))
            case OrF(p, q):
                return '({} || {})'.format(
                    prettyprint(p),
                    prettyprint(q))
            case ImpliesF(p, q):
                return '{} -> ({})'.format(
                    prettyprint(p),
                    prettyprint(q))
        
    elif isinstance(o, Prog):
        match o:
            case Asgn(left, right):
                return '{}{} := {}'.format(ids,
                                           prettyprint(left, indent=0), 
                                           prettyprint(right, indent=0))

            case Assert(Q):
                return '{}assert({})'.format(ids,
                                             prettyprint(Q, indent=0))

            case Seq(alpha, beta):
                return '{};\n{}'.format(prettyprint(alpha, indent=indent),
                                        prettyprint(beta, indent=indent))

            case If(Q, alpha, beta):
                return '{}if({}) {{\n{}{}\n{}}} {{\n{}{}\n{}}}'.format(
                    ids,
                    prettyprint(Q, indent=0),
                    ids,
                    prettyprint(alpha, indent=indent+4),
                    ids,
                    ids,
                    prettyprint(beta, indent=indent+4),
                    ids)

            case While(Q, alpha):
                return '{}while({}) {{\n{}\n{}}}'.format(
                    ids,
                    prettyprint(Q, indent=0),
                    prettyprint(alpha, indent=indent+4),
                    ids)
    else:
        return str(o)

The following example represents a Fibonacci generator as a `Prog`

In [7]:
fib_init = Seq(Asgn(Var('x'), Const(0)),
               Seq(Asgn(Var('y'), Const(1)),
                   Seq(Asgn(Var('z'), Const(1)),
                       Asgn(Var('i'), Const(0)))))
fib_loop = While(LtF(Var('i'), Var('n')),
                 Seq(Asgn(Var('x'), Var('y')),
                     Seq(Asgn(Var('y'), Var('z')),
                         Seq(Asgn(Var('z'), Sum(Var('x'), Var('y'))),
                             Asgn(Var('i'), Sum(Var('i'), Const(1)))))))
fib = Seq(fib_init, fib_loop)
print(prettyprint(fib))

x := 0;
y := 1;
z := 1;
i := 0;
while(i < n) {
    x := y;
    y := z;
    z := x + y;
    i := i + 1
}


Below is our implementation of the box modality. This is refined a bit from what we did live at the end of lecture.
* Formulas are simplified aggressively at each step
* We use a slightly less complicated version of the axiom for loops

In [8]:
# Apply axioms of dynamic logic for [alpha] P
def box(alpha: Prog, P: BoolRef, max_depth: int=10) -> BoolRef:
    
    if max_depth < 1:
        return BoolVal(False)
    
    match alpha:
        # [x := e] P(x) <--> P(e)
        case Asgn(left, right):
            return simplify(substitute(P, [(term_enc(left), term_enc(right))]))
        
        # [assert(Q)] P <--> P ^ Q
        case Assert(Q):
            return simplify(And(fmla_enc(Q), P))
            
        # [alpha; beta] P <--> [alpha]([beta] P)
        case Seq(alpha_p, beta_p):
            return simplify(box(alpha_p, 
                                box(beta_p, P, max_depth), 
                                max_depth))
            
        # [If(Q) alpha else beta] P <--> (Q -> [alpha] P) ^ (~Q -> [beta] P)
        case If(Q, alpha_p, beta_p):
            return simplify(And(Implies(fmla_enc(Q), box(alpha_p, P, max_depth)),
                                Implies(fmla_enc(NotF(Q)), box(beta_p, P, max_depth))))
            
        # [while(Q) alpha] P <--> (Q -> [alpha; while(Q) alpha] P) ^ (~Q -> P)
        # Note that this is equivalent to:
        # [while(Q) alpha] P <--> [if(Q) { alpha; while(Q) alpha } else { assert(True) }] P
        case While(Q, alpha_p):
            return simplify(box(If(Q, 
                                   Seq(alpha_p, alpha),
                                   Assert(TrueC(None))), 
                                P, max_depth-1))

A basic sanity check: `[x := 1](x < 0)`

We should get `1 < 0`, which simplifies to `False`

In [9]:
alpha = Asgn(Var('x'), Const(1))
post = LtF(Var('x'), Const(0))
pre = box(alpha, fmla_enc(post))
print('Program:', prettyprint(alpha))
print('Postcondition:', prettyprint(post))
print('Verification condition:', pre)

Program: x := 1
Postcondition: x < 0
Verification condition: False


A slightly more interesting example: is 88 a Fibonacci number? If so, then the formula generated by `box` for the postcondition `x = 88` should be satisfiable.

We see that it is not, because the formula is unsatisfiable, which means that there are no inputs (i.e., assignments to `n`) that result in `x = 88` in the final state.

In [10]:
P = fmla_enc(EqF(Var('x'), Const(88)))
vc = box(fib, P, 100)
s = Solver()
s.add(vc)
s.check()

unsat

By the same token, we can test valid Fibonacci numbers for their index: `box` essentially runs the Fibonacci program in reverse to find the initial states that yield the final states described in the postcondition.

In [11]:
P = fmla_enc(EqF(Var('x'), Const(89)))
vc = box(fib, P, 100)
s = Solver()
s.add(vc)
s.check()

sat

We can see what satisfying assignment the solver found.

In [12]:
s.model()

[n = 11]

Using `box`, we can also implement a (very) simple contract verifier.
The function below returns `True` if the program satisfies its contract, and if it doesn't, then it returns a counterexample input that will violate the contract.

Note the way that it works: `z3` is a *satisfiability* solver, not a validity checker. We've already seen that these semantic notions are duals of eachother, so to check for the validity of `P -> [alpha] Q` using `z3`, we negate and check for satisfiability. This negated formula is called a *verification condition*. It is unsatisfiable iff the program follows its contract, and otherwise any satisfying assignments correspond to an initial state (i.e., set of inputs) that will lead the program to violate its contract.

This type of verification is called *bounded model checking*.

In [13]:
def verify_contract(alpha: Prog, P: Formula, Q: formula, max_depth=10):
    vc = Not(Implies(fmla_enc(P), 
                     box(alpha, fmla_enc(Q), max_depth=max_depth)))
    s = Solver()
    s.add(vc)
    if s.check() == unsat:
        return True
    else:
        return s.model()

The precondition below encodes that we should find the fourth Fibonacci number (0-based), and the postcondition asserts that the result should be 1. This is incorrect, as the result should be 2. The verifier should give us a counterexample.

In [14]:
pre = EqF(Var('n'), Const(3))
post = EqF(Var('x'), Const(1))
verify_contract(fib, pre, post)

[n = 3]

Correcting the mistake, we see that the verifier behaves as expected.

In [15]:
pre = EqF(Var('n'), Const(2))
post = EqF(Var('x'), Const(1))
verify_contract(fib, pre, post)

True